In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive"

'Archive 2.zip'       CycleGAN.ipynb	     horse2zebra   results.csv
 Assignment_2.ipynb   DiscoGan.ipynb	     Image_1
'Colab Notebooks'    'Getting started.pdf'   Image_2


In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from PIL import Image
import random

#########################################################################################################
######################################## HELPER FUNCTIONS ###############################################
#########################################################################################################

In [0]:

class helper_functions:
    
    
    def conv_ops_of_Gen_and_Disc(self, data, num_filters, filter_size, padding,activation_fn, wts_initializer,
                                    bias_initializer, normalizer_fn, batch_size, is_Gen = True):
        
        stride = 2
        if is_Gen:
            last_stride = 1
            last_activation = tf.nn.relu
        else:
            last_stride = 2
            last_activation = activation_fn
            
            
        conv_1 = tf.contrib.layers.conv2d(inputs = data, num_outputs = num_filters[0], 
                                             kernel_size = filter_size, stride = stride,
                                             padding = padding, activation_fn = activation_fn,
                                             weights_initializer = wts_initializer, scope = 'conv_1', 
                                             biases_initializer = bias_initializer)
        
        conv_2 = tf.contrib.layers.conv2d(inputs = conv_1, num_outputs = num_filters[1], 
                                             kernel_size = filter_size, stride = stride,
                                             padding = padding, activation_fn = activation_fn,
                                             weights_initializer = wts_initializer, scope = 'conv_2', 
                                             biases_initializer = bias_initializer, normalizer_fn = normalizer_fn)
        
        conv_3 = tf.contrib.layers.conv2d(inputs = conv_2, num_outputs = num_filters[2], 
                                             kernel_size = filter_size, stride = stride,
                                             padding = padding, activation_fn = activation_fn,
                                             weights_initializer = wts_initializer, scope = 'conv_3', 
                                             biases_initializer = bias_initializer, normalizer_fn = normalizer_fn)
        
        
        conv_4 = tf.contrib.layers.conv2d(inputs = conv_3, num_outputs = num_filters[3], 
                                             kernel_size = filter_size, stride = stride,
                                             padding = padding, activation_fn = activation_fn,
                                             weights_initializer = wts_initializer, scope = 'conv_4', 
                                             biases_initializer = bias_initializer, normalizer_fn = normalizer_fn)
        
        conv_5 = tf.contrib.layers.conv2d(inputs = conv_4, num_outputs = num_filters[4], 
                                             kernel_size = filter_size, stride = last_stride,
                                             padding = padding, activation_fn = last_activation,
                                             weights_initializer = wts_initializer, scope = 'conv_5', 
                                             biases_initializer = bias_initializer, normalizer_fn = normalizer_fn)
        
        if is_Gen:
            return tf.reshape(conv_5, [-1, 8, 8, 128])
        
        
        else:
            conv_5_flatten = tf.reshape(conv_5, [-1, 2 * 2 * 512])
            fc_1 = tf.contrib.layers.fully_connected(inputs = conv_5_flatten, num_outputs = 512, 
                                                     activation_fn = activation_fn, 
                                                     weights_initializer = wts_initializer, scope = "fc_1", 
                                                     normalizer_fn = normalizer_fn)
            fc_2 = tf.contrib.layers.fully_connected(inputs = fc_1, num_outputs = 1, 
                                                     activation_fn = tf.nn.sigmoid, 
                                                     weights_initializer = wts_initializer, scope = "fc_2", 
                                                     normalizer_fn = normalizer_fn)
            return fc_2
            

#########################################################################################################
######################################## GENERATOR ######################################################
#########################################################################################################

In [0]:

class Generator:
    def __init__(self, name, helper_functions, batch_size):
        self.gen_name         = name
        self.filter_size      = 4
        self.enc_num_filters  = [32, 64, 128, 256, 512]
        self.dec_num_filters  = [256, 128, 64, 3]
        self.enc_stride       = 2
        self.dec_stride       = 1
        self.padding          = 'SAME'
        self.enc_activation   = tf.nn.leaky_relu
        self.dec_activation   = tf.nn.relu
        self.wts_initializer  = tf.truncated_normal_initializer(stddev=0.02)
        self.bias_initializer = tf.zeros_initializer()
        self.normalizer_fn    = tf.contrib.layers.batch_norm
        self.helper           = helper_functions
        self.batch_size       = batch_size
        
        
    #################################### Encoding part #################################################
    
    def Gen_encoder(self, data):
        
        return self.helper.conv_ops_of_Gen_and_Disc(data, self.enc_num_filters, self.filter_size, self.padding, 
                                                    self.enc_activation, self.wts_initializer, self.bias_initializer, 
                                                    self.normalizer_fn, self.batch_size, True)

    
    
    
    
    
    #################################### Decoding part #################################################
    
    def Gen_decoder(self, data):

        dec_conv_1 = tf.contrib.layers.conv2d(inputs = data, num_outputs = self.dec_num_filters[0], 
                                             kernel_size = self.filter_size, stride = self.dec_stride,
                                             padding = self.padding, activation_fn = self.dec_activation,
                                             weights_initializer = self.wts_initializer, scope = 'dec_conv_1', 
                                             biases_initializer = self.bias_initializer, normalizer_fn = self.normalizer_fn)
        dec_conv_1 = tf.reshape(dec_conv_1, [-1, 16, 16, 64])
  

        #####
        dec_conv_2 = tf.contrib.layers.conv2d(inputs = dec_conv_1, num_outputs = self.dec_num_filters[1], 
                                             kernel_size = self.filter_size, stride = self.dec_stride,
                                             padding = self.padding, activation_fn = self.dec_activation,
                                             weights_initializer = self.wts_initializer, scope = 'dec_conv_2', 
                                             biases_initializer = self.bias_initializer, normalizer_fn = self.normalizer_fn)
        dec_conv_2 = tf.reshape(dec_conv_2, [-1, 32, 32, 32])
        

        #####
        dec_conv_3 = tf.contrib.layers.conv2d(inputs = dec_conv_2, num_outputs = self.dec_num_filters[2], 
                                             kernel_size = self.filter_size, stride = self.dec_stride,
                                             padding = self.padding, activation_fn = self.dec_activation,
                                             weights_initializer = self.wts_initializer, scope = 'dec_conv_3', 
                                             biases_initializer = self.bias_initializer, normalizer_fn = self.normalizer_fn)
        dec_conv_3 = tf.reshape(dec_conv_3, [-1, 64, 64, 16])
        

        #####
        dec_conv_4 = tf.contrib.layers.conv2d(inputs = dec_conv_3, num_outputs = self.dec_num_filters[3], 
                                             kernel_size = self.filter_size, stride = self.dec_stride,
                                             padding = self.padding, activation_fn = self.dec_activation,
                                             weights_initializer = self.wts_initializer, scope = 'dec_conv_4', 
                                             biases_initializer = self.bias_initializer, normalizer_fn = self.normalizer_fn)
        
        return dec_conv_4
    
    
    
    #################################### Feed forward #################################################
    
    def gen_feed_forward(self, data):
        with tf.variable_scope(self.gen_name, reuse = tf.AUTO_REUSE):
            encoding_res = self.Gen_encoder(data)
            decoding_res = self.Gen_decoder(encoding_res)
            
            self.gen_variables     = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.gen_name)
            return decoding_res

#########################################################################################################
######################################## DISCRIMINATOR ##################################################
#########################################################################################################

In [0]:
class Discriminator:
    def __init__(self, name, helper_functions, batch_size):
        self.disc_name        = name
        self.filter_size      = 4
        self.num_filters      = [32, 64, 128, 256, 512]
        self.activation_fn    = tf.nn.leaky_relu
        self.wts_initializer  = tf.truncated_normal_initializer(stddev=0.02)
        self.bias_initializer = tf.zeros_initializer()
        self.normalizer_fn    = tf.contrib.layers.batch_norm
        self.helper           = helper_functions
        self.padding          = 'SAME'
        self.batch_size       = batch_size
        
    #################################### Feed forward #################################################
    
    def disc_feed_forward(self, data):
        with tf.variable_scope(self.disc_name, reuse = tf.AUTO_REUSE):
            disc_res = self.helper.conv_ops_of_Gen_and_Disc(data, self.num_filters, self.filter_size, self.padding, 
                                                    self.activation_fn, self.wts_initializer, self.bias_initializer, 
                                                    self.normalizer_fn, self.batch_size, False)
    
            self.disc_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.disc_name)
            return disc_res
    

#########################################################################################################
######################################## DATA PROCESSING ################################################
#########################################################################################################

In [0]:
class Data_reading_and_processing:
    def __init__(self, data_X_dir, data_Y_dir, val_X_dir, val_Y_dir):

        self.new_data_X_dir       = data_X_dir # '/Users/vijay/Downloads/new_bags/train/'
        self.new_data_Y_dir       = data_Y_dir #'/Users/vijay/Downloads/new_shoes/train/'
        self.new_val_X_dir        = val_X_dir  # '/Users/vijay/Downloads/new_bags/val/'
        self.new_val_Y_dir        = val_Y_dir  # '/Users/vijay/Downloads/new_shoes/val/'
        self.train_X_images_names = os.listdir(self.new_data_X_dir)
        self.train_Y_images_names = os.listdir(self.new_data_Y_dir)
        self.val_X_images_names   = os.listdir(self.new_val_X_dir)
        self.val_Y_images_names   = os.listdir(self.new_val_Y_dir)
        

        
    
    ########################################################################
    
    def crop_and_save_images_into_new_dir(self, prev_path, new_path, images_names):
    
        for image_name in images_names:
            image = Image.open(prev_path + image_name)
            image = image.resize([128,64])
            image = image.crop([64,0,128,64])
            image.save(new_path + image_name)

        
        
    ########################################################################
    
    def get_batch(self, index, images_names, path, batch_size):

        images_arr = []
        images_batch = images_names[index : index + batch_size]
        for image_name in images_batch:
            
            image = cv2.resize(cv2.imread(path + image_name), (64, 64))
#             img = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            img = cv2.normalize(image, None, 0, 128, cv2.NORM_MINMAX)
            images_arr.append(img)
        return np.asarray(images_arr)
    
    
    ##########################################################################
    # only for testing
    def get_input_batch(self, index, images_names, path, batch_size):
        images_arr = []
        images_batch = images_names[index : index + batch_size]
        for image_name in images_batch:
            
            image = cv2.resize(cv2.imread(path + image_name), (128, 64))
            img = cv2.normalize(image, None, 0, 128, cv2.NORM_MINMAX)
            img = img[0 : 64, 64 : 128]
            images_arr.append(img)

        return np.asarray(images_arr)


#########################################################################################################
########################################   DISCO GAN   ##################################################
#########################################################################################################

In [0]:
class DISCO_GAN:
    def __init__(self, image_shape, batch_size, learning_rate, num_epochs, to_restore, 
                                                   helper_functions, output_dir, data_object):
        self.image_shape = image_shape
        self.batch_size  = batch_size
        self.lr          = learning_rate

        self.num_epochs  = num_epochs
        self.to_restore  = to_restore
        self.helper      = helper_functions
        self.output_dir  = output_dir  # '/Users/vijay/Downloads/DiscoGAN/test/'
        self.X           = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], self.image_shape[2]])
        self.Y           = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], self.image_shape[2]])
        self.data_object = data_object
        
        self.check_point_dir = self.output_dir + 'checkpoints/'
        self.global_step     = tf.Variable(0, name = 'global_step', trainable = False)

    ########################################################################
    
    def get_optimizer(self):
        
        with tf.variable_scope('optim', reuse = tf.AUTO_REUSE):
            
            Gen_XY = Generator('Gen_XY', self.helper, self.batch_size) 
            Gen_YX = Generator('Gen_YX', self.helper, self.batch_size) 
            
            Disc_X = Discriminator('Disc_X', self.helper, self.batch_size)
            Disc_Y = Discriminator('Disc_Y', self.helper, self.batch_size)
            
            '''
            naming conventions:
                    gen_source_(result)
                    
                    gen_x_fake_y
                    
                        source = x
                        result = fake_y
                    
                    gen_y_fake_x 
                    
                        source = y
                        result = fake_x
                    
                    gen_fake_y_fake_x
                    
                        source = fake_y
                        result = fake_x
                    
                    gen_fake_x_fake_y
                    
                        source = fake_x
                        result = fake_y
            '''
            self.gen_x_fake_y = Gen_XY.gen_feed_forward(self.X) # constructs fake_Y images from X
            self.gen_y_fake_x = Gen_YX.gen_feed_forward(self.Y) # constructs fake_X images from Y
            
            gen_fake_y_fake_x = Gen_YX.gen_feed_forward(self.gen_x_fake_y) # reconstructs X (fake) from fake_Y images
            gen_fake_x_fake_y = Gen_XY.gen_feed_forward(self.gen_y_fake_x) # reconstructs Y (fake) from fake_X images
            
            disc_fake_x = Disc_X.disc_feed_forward(self.gen_y_fake_x) # discriminates fake_x generated by gen_yx from real Y
            disc_fake_y = Disc_Y.disc_feed_forward(self.gen_x_fake_y) # discriminates fake_y generated by gen_xy from real X
            
            disc_real_x = Disc_X.disc_feed_forward(self.X)
            disc_real_y = Disc_Y.disc_feed_forward(self.Y)
            
            #####**********************************************
            
            # Discrimator loss
            loss_disc_real_x = tf.reduce_mean(tf.squared_difference(disc_real_x, 1))
            loss_disc_real_y = tf.reduce_mean(tf.squared_difference(disc_real_y, 1))

            loss_disc_fake_x = tf.reduce_mean(tf.square(disc_fake_x))
            loss_disc_fake_y = tf.reduce_mean(tf.square(disc_fake_y))

            self.disc_X_total_loss = (loss_disc_real_x + loss_disc_fake_x) / 2
            self.disc_Y_total_loss = (loss_disc_real_y + loss_disc_fake_y) / 2
            
            # Reconstruction loss
            reconstruction_loss_x = tf.reduce_mean(tf.losses.mean_squared_error(self.X, gen_fake_y_fake_x))
            reconstruction_loss_y = tf.reduce_mean(tf.losses.mean_squared_error(self.Y, gen_fake_x_fake_y))
            total_reconstruction_loss = reconstruction_loss_x + reconstruction_loss_y
            
            # Generator loss
            '''
            Generator should be successful in fooling the discriminator. In other words, Generator should make the discriminator
            believe that the generated images are real images. This can be done if the recommendation made by the Discriminator
            is as close to 1 as possible. so, Generator would like to minimize ((Discriminator_X(Generator_YX(image))) - 1)^2
            '''
            gen_loss_fake_x = tf.reduce_mean(tf.squared_difference(disc_fake_x, 1))
            gen_loss_fake_y = tf.reduce_mean(tf.squared_difference(disc_fake_y, 1))
            
            self.total_gen_XY_loss = (10 * total_reconstruction_loss) + gen_loss_fake_y
            self.total_gen_YX_loss = (10 * total_reconstruction_loss) + gen_loss_fake_x
#             gen_loss    = 10 * total_reconstruction_loss + (gen_loss_fake_x + gen_loss_fake_y)
            disc_loss   = self.disc_X_total_loss + self.disc_Y_total_loss
            
            #####**********************************************
            
            
            trainable_variables = tf.trainable_variables()
            Gen_xy_variables = [var for var in trainable_variables if 'Gen_XY' in var.name]
            Gen_yx_variables = [var for var in trainable_variables if 'Gen_YX' in var.name]
            Disc_x_variables = [var for var in trainable_variables if 'Disc_X' in var.name]
            Disc_y_variables = [var for var in trainable_variables if 'Disc_Y' in var.name]
            
            
            
            # Optimizers
            self.Gen_XY_optim = tf.train.AdamOptimizer(learning_rate = self.lr, beta1 = 0.5).minimize(self.total_gen_XY_loss, var_list = Gen_xy_variables)
            self.Gen_YX_optim = tf.train.AdamOptimizer(learning_rate = self.lr, beta1 = 0.5).minimize(self.total_gen_YX_loss, var_list = Gen_yx_variables)
            self.Disc_X_optim = tf.train.AdamOptimizer(learning_rate = self.lr, beta1 = 0.5).minimize(self.disc_X_total_loss, var_list = Disc_x_variables)
            self.Disc_Y_optim = tf.train.AdamOptimizer(learning_rate = self.lr, beta1 = 0.5).minimize(self.disc_Y_total_loss, var_list = Disc_y_variables)
            
        
        
        
        
    ########################################################################
    
    def train_Disco_GAN(self):
        count = 0
        print('training the Disco GAN')
        self.get_optimizer()
        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        
        new_train_X_images_path = self.data_object.new_data_X_dir
        new_train_Y_images_path = self.data_object.new_data_Y_dir
        
        num_of_train_X_images = len(self.data_object.train_X_images_names)
        num_of_train_Y_images = len(self.data_object.train_Y_images_names)
        num_of_train_images   = min(num_of_train_X_images, num_of_train_Y_images, 1060)
        
        
        if self.to_restore:
          
          checkpoint_file = tf.train.latest_checkpoint(self.check_point_dir)
          saver.restore(self.session, checkpoint_file)
            
        if not os.path.exists(self.check_point_dir):
            os.makedirs(self.check_point_dir)
        
        for epoch in range(self.session.run(self.global_step), num_epochs):
             
            random.shuffle(self.data_object.train_X_images_names)
            random.shuffle(self.data_object.train_Y_images_names)
            count = 0
            with tf.device('/gpu:0'):
              for index in range(0, num_of_train_images, batch_size):
#                   if count < 1:
                    train_x = self.data_object.get_batch(index, self.data_object.train_X_images_names, 
                                                                        new_train_X_images_path, self.batch_size)
                    train_y = self.data_object.get_batch(index, self.data_object.train_Y_images_names, 
                                                                       new_train_Y_images_path, self.batch_size)

#                       if count == 0:
#                           for arr in train_x:
#                               (Image.fromarray(arr, 'RGB')).show()


                     # training Gen_XY_optim to generate fake Y images from true X
                    _, fake_Y = self.session.run([self.Gen_XY_optim, self.gen_x_fake_y], feed_dict = {self.X : train_x,
                                                                                           self.Y : train_y})

                    # training Gen_YX_optim to generate fake X images from true Y
                    _, fake_X = self.session.run([self.Gen_YX_optim, self.gen_y_fake_x], feed_dict = {self.X : train_x,
                                                                                               self.Y : train_y})
                    _ = self.session.run([self.Disc_X_optim], feed_dict = {self.X : train_x,
                                                                          self.Y : train_y})

                    _ = self.session.run([self.Disc_Y_optim], feed_dict = {self.X : train_x,
                                                                          self.Y : train_y})
                    count = count + 1

#                   else:
#                       break

              self.session.run(tf.assign(self.global_step, epoch + 1))
              print('epoch ' + str(epoch) + ' samples ' + str(count))
              if epoch % 100 == 0 and epoch >= 50:
                saver.save(self.session,self.check_point_dir + 'DiscoGAN', global_step = epoch)

    
    
    
    
    ########################################################################
    
    def create_directory(self, directory):
        if not os.path.exists(directory):
            os.makedirs(directory)
                
    
    
    
    ########################################################################
    
    def apply_transformtions_on_test_images(self):
        print('testing the test images')
        count = 0

        
        fake_X_path = self.output_dir + 'fake_X/'
        fake_Y_path = self.output_dir + 'fake_Y/'
        val_X_path  = self.output_dir + 'val_X/'
        val_Y_path  = self.output_dir + 'val_Y/'
        

        self.create_directory(fake_X_path)
        self.create_directory(fake_Y_path)
        self.create_directory(val_X_path)
        self.create_directory(val_Y_path)
        
        val_X_images_path = self.data_object.new_val_X_dir
        val_Y_images_path = self.data_object.new_val_Y_dir

        
        val_X_images_names = self.data_object.val_X_images_names
        val_Y_images_names = self.data_object.val_Y_images_names
        
        num_of_test_images = min(50, len(val_X_images_names), len(val_Y_images_names))
        with tf.Session() as session:
            meta_graph = tf.train.import_meta_graph(self.check_point_dir + 'DiscoGAN-2000.meta')
            meta_graph.restore(session, tf.train.latest_checkpoint(self.check_point_dir))
            
            random.shuffle(val_X_images_names)
            random.shuffle(val_Y_images_names)
            
            for index in range(0, num_of_test_images, self.batch_size):
                val_x = self.data_object.get_batch(index, val_X_images_names, val_X_images_path, self.batch_size)
                val_y = self.data_object.get_batch(index, val_Y_images_names, val_Y_images_path, self.batch_size)
                
                

                fake_X, fake_Y = session.run([self.gen_y_fake_x, self.gen_x_fake_y], feed_dict = {self.X : val_x,
                                                                                                   self.Y : val_y})
                
                for image_no in range(0, len(val_x)):
                    cv2.imwrite((fake_X_path + str(count) + '_fx.png'), np.uint8(fake_X[image_no]))
                    cv2.imwrite((fake_Y_path + str(count) + '_fy.png'), np.uint8(fake_Y[image_no]))
                    cv2.imwrite((val_X_path + str(count) + '_tx.png'), np.uint8(val_x[image_no]))
                    cv2.imwrite((val_Y_path + str(count) + '_ty.png'), np.uint8(val_y[image_no]))
                    count = count + 1

In [0]:

if __name__ == '__main__':
    image_shape = [64, 64, 3]
    batch_size = 10
    learning_rate = 0.0002
    num_epochs = 2001
    to_restore = True
    helper = helper_functions()
    if not os.path.exists('/content/drive/My Drive/DiscoGAN'):
        os.makedirs('/content/drive/My Drive/DiscoGAN')
    output_dir  = '/content/drive/My Drive/DiscoGAN/test/'
    
    ### data object
    data_X_path = '/content/drive/My Drive/horse2zebra/trainA/' # X
    data_Y_path = '/content/drive/My Drive/horse2zebra/trainB/'   # Y
    val_X_path = '/content/drive/My Drive/horse2zebra/testA/'
    val_Y_path = '/content/drive/My Drive/horse2zebra/testB/'
#     new_images_dir = '/Users/vijay/Downloads/DiscoGAN/new/'
    data_object = Data_reading_and_processing(data_X_path, data_Y_path, val_X_path, val_Y_path)
    disco_GAN = DISCO_GAN(image_shape, batch_size, learning_rate, num_epochs, to_restore, helper, output_dir, data_object)
    disco_GAN.train_Disco_GAN()
    disco_GAN.apply_transformtions_on_test_images()
    
    
   